In [ ]:
import numpy as np
import re

In [ ]:
f = open("book-war-and-peace.txt")
t = f.readlines()
t = ''.join(t)

In [ ]:
text = re.sub('[^a-zA-Z]', ' ', t)[:50000]
text = text.lower()

In [ ]:
d = {}
for i in range(0, 26):
    d[chr(i+97)] = i
d[' '] = 26

In [ ]:
V = np.zeros(50000)
for i in range(len(text)):
    V[i] = d[text[i]]

In [ ]:
V = V.astype(int)
print(V)

In [ ]:
# Transition Probabilities
a = np.array([[0.47468,0.52532],[0.51656,0.48344]])
print(a)

In [ ]:
b = np.array([[0.03735,0.03909 ],
[0.03408 ,0.03537 ],
[0.03455 ,0.03537 ],
[0.03828, 0.03909 ],
[0.03782, 0.03583 ],
[0.03922 ,0.03630 ],
[0.03688, 0.04048 ],
[0.03408, 0.03537 ],
[0.03875 ,0.03816 ],
[0.04062 ,0.03909 ],
[0.03735 ,0.03490 ],
[0.03968, 0.03723 ],
[0.03548 ,0.03537 ],
[0.03735 ,0.03909 ],
[0.04062, 0.03397 ],
[0.03595, 0.03397 ],
[0.03641, 0.03816 ],
[0.03408, 0.03676 ],
[0.04062 ,0.04048 ],
[0.03548, 0.03443 ],
[0.03922, 0.03537 ],
[0.04062 ,0.03955],
[0.03455, 0.03816 ],
[0.03595, 0.03723 ],
[0.03408,0.03769 ],
[0.03408 ,0.03955 ],
[0.03688, 0.03397 ]])
b = b.T
print(b.shape)

In [ ]:
# Equal Probabilities for the initial distribution
initial_distribution = np.array([0.51316,0.48684])
print(initial_distribution)

In [ ]:
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]

    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            #                  ((1x2) . (1x2))      *     (1)
            #                        (1)            *     (1)
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]

    return alpha

def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))

    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])

    return beta

def baum_welch(V, a, b, initial_distribution, n_iter):
    M = a.shape[0]
    T = len(V)

    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)

        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)

        b = np.divide(b, denominator.reshape((-1, 1)))

    return {"a":a, "b":b}

In [ ]:
print(V.shape)
print(a.shape)
print(b.shape)
print(V.shape[0], a.shape[0])

In [ ]:
print(baum_welch(V, a, b, initial_distribution, n_iter=2))